## Chapter 20. 속성 디스크립터
### 20.1 디스크립터 예: 속성 검증
프로퍼티 팩토리를 사용하면 함수형 프로그래밍 스타일을 적용함으로써 똑같은 게터와 세터를 반복해서 구현할 필요가 없다. 프로퍼티 함수는 고위 함수로서 일련의 접근자 함수를 매개변수화하고 storage_name과 같은 환경 변수를 클로저에 담아서 사용자 정의 프로퍼티 객체를 생성한다. 이와 동일한 문제를 객체지향방식으로 해결한 것이 디스크립터 클래스이다

주요 용어는 아래과 같다.
<p>
    <li>
        <strong>디스크립터 클래스</strong><br/>
        디스크립터 프로토콜을 구현하는 클래스. Quantity 클래스가 디스크립터 클래스이다.
    </li>
</p>
<p>
    <li>
        <strong>관리 대상 클래스</strong><br/>
        디스크립터 객처를 클래스 속성으로 선언하는 클래스. LineItem 클래스가 관리 대상 클래스이다.
    </li>
</p>
<p>
    <li>
        <strong>디스크립터 객체</strong><br/>
        관리 대상 클래스의 클래스 속성으로 선언된 디스크립터 클래스의 객체. [그림 20-1]에서 각각의 디스크립터 객체는 밑줄 친 이름을 가진 구성 화살표로 표현된다.(UML에서 밑줄 친 속성은 클래스 속성을 나타낸다.) 디스크립터 객체를 가진 LineItem 클래스 쪽에 검은 마름모가 연결된다.
    </li>
</p>
<p>
    <li>
        <strong>관리 대상 객체</strong><br/>
        관리 대상 클래스의 객체. 이 예제에서는 LineItem 클래스의 객체들이 관리 대상 객체가 된다.(클래스 다이어그램에는 나타나 있지 않다.)
    </li>
</p>
<p>
    <li>
        <strong>저장소 속성</strong><br/>
            관리 대상 객체 안의 관리 대상 속성값을 담을 속성. [그림 20-1]에서 LineItem 객체의 weight와 price 속성이 저장소 속성이다. 이들은 디스크립터 객체와는 별개의 속성으로 향상 클래스 속성이다.
    </li>
</p>
<p>
    <li>
        <strong>관리 대상 속성</strong><br/>
            디스크립터 객체에 의해 관리되는 관리 대상 클래스 안의 공개 속성으로 이 속성의 값은 저장소 속성에 저장된다. 즉, 디스크립터 객체와 저장소 속성이 관리 대상 속성에 대한 기반을 제공한다.
    </li>
</p>

In [1]:
""" [예제 20-1] bulkfood_v3.py: LineItem의 속성을 관리하는 Quantity 디스크립터 """

class Quantity: # 상속은 필요없다.
    def __init__(self, storage_name):
        self.storage_name = storage_name # 관리대상 객체에서 값을 보관할 속성의 이름이다.
    
    def __set__(self, instance, value):
        """
        관리 대상 속성에 값을 할당할 때 호출된다.
        - self : 디스크립트 객체, 여기서는 LineItem.weight, LineItem.price에 대입하는 객체
        - instance : 관리 대상 객체, 여기서는 LintItem
        - value : 할당할 값
        """
        print(repr(self))
        print(repr(instance))
        if value > 0:
            instance.__dict__[self.storage_name] = value # 관리 대상 객체의 __dict__를 직접 처리한다. 
                                                         # setattr() 내장함수를 사용하면 또 다시 __set__() 메서드가 호출되어 무한 재귀가 된다.
        else:
            raise ValueError('value must be > 0')
            
class LineItem:
    weight = Quantity('weight')
    price = Quantity('price')
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [2]:
truffle = LineItem('White truffle', 100, 0)

ValueError: value must be > 0

```
1) instance.__dict__[self.storage_name] = value
2) self.__dict__[self.storage_name] = value
```
2)로의 변경은 문제가 있다. 여기서 self는 디스크립터 객체로서, 관리 대상 클래스의 클래스 속성이다. 메모리에 수천 개의 LineItem 객체가 있더라도 디스크립터 객체는 LineItem.weight, LineItem.price, 단 두 개 밖에 없다. 따라서 디스크립터 객체에 저장하는 모든 것은 LineItem 클래스 속성이 되어, 모든 LineItem 객체가 공유한다.

[예제 20-1]의 단점은 관리 대상 클래스 본체에 디스크립터 개게를 생성할 때 속성명을 반복해야 한다는 것이다. 아래와 같으면 더 좋을 것이다. 
```
class LineItem:
    weight = Quantity()
    price = Quantity()
```

### 20.1.2 LineItem 버전 #4: 자동 저장소 속성명
디스크립터를 선언할 때 속성명을 반복 입력하지 않기 위해 각 Quantity 객체의 storage_name에 대한 고유한 문자열을 생성할 것이다. storage_name을 생성하기 위해 '\_Quantity#' 문자열 뒤에 정수를 연결한다. 새로운 Quality 디스크립터 객체가 클래스에 연결될 때마다 Quantity.\_\_counter 클래스 속성의 값이 증가한다. 해시기호(#)를 사용하면 올바른 파이썬 문법이 아니기 때문에 점 표기법으로 생성한 속성과 충돌되지 않도록 보장할 수 있다.

여기서는 instance.\_\_dict\_\_ 대신 getattr()와 setattr() 내장함수를 이용해서 값을 저장할 수 있었다. 관리 대상 속성과 저장소 속성의 이름이 다르기 때문에 저장소 속성에 getattr()을 호출하더라도 디스크립터를 실행하지 않으므로, [예제 20-1]의 무한 재귀가 일어나지 않는다.

In [3]:
""" [예제 20-2] bulkfood_v4.py: 각 Quantity 디스크립터는 고유한 storage_name을 가진다. """

class Quantity:
    __counter = 0 # Quantity 객체의 수를 센다.
    
    def __init__(self):
        cls = self.__class__  # Quantity 클래스에 대한 참조
        prefix = cls.__name__ 
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index) # 고유한 이름 (예를 들면_Quantity#0)을 만든다. 
        cls.__counter += 1
    
    def __get__(self, instance, owner): # 관리 대상 속성의 이름이 storage_name과 동일하지 않으므로 __get__() 메서드를 구현현다.
        return getattr(instance, self.storage_name) # instance에서 값을 가져온다.
    
    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be 0')
            
class LineItem:
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weigth * self.price

In [4]:
coconuts = LineItem('Brazilian', 20, 17.95)

In [5]:
coconuts.weight, coconuts.price

(20, 17.95)

In [6]:
getattr(coconuts, '_Quantity#0'), getattr(coconuts, '_Quantity#1')

(20, 17.95)

In [7]:
LineItem.weight

AttributeError: 'NoneType' object has no attribute '_Quantity#0'

\_\_get\_\_() 함수의 owner 인수는 관리 대상 클래스(즉, LienItem)에 대한 참조이며 디스크립터를 이용해서 클래스 속성을 가져올 때 유용하게 사용할 수 있다. LineItem.weight 처럼 글래수에서 관리 대상 속성을 가져올 때는 디스크립터 \_\_get\_\_() 메서드가 instance 인수값으로 None을 받는다. 따라서 위처럼 Attribute가 발생하게 되지만, Nonetype과 _Quantity#0 등 내부 구현에 관련된 혼란스러운 메시지는 삭제하는 것이 좋다. 아래는 이를 개선한 것이다.

In [8]:
""" [예제 20-3] bulkfood_v4.py: 관리되는 클래스를 통해 호출되면 get()은 디스크립터 자체에 대한 참조를 반환한다. """

class Quantity:
    __counter = 0 # Quantity 객체의 수를 센다.
    
    def __init__(self):
        cls = self.__class__  # Quantity 클래스에 대한 참조
        prefix = cls.__name__ 
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index) # 고유한 이름 (예를 들면_Quantity#0)을 만든다. 
        cls.__counter += 1
    
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
    
    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be 0')